# Bisect Halveringssøk i Python og Julia


In [7]:
import time

In [16]:
import sqlite3
import math
import pandas as pd

In [4]:
db = "/mnt/disk1/Github/total.db"

In [18]:
def bisect(db, word):
    minrow = query(db, "select rowid from words order by rowid asc limit 1")[0][0]
    maxrow = query(db, "select rowid from words order by rowid desc limit 1")[0][0]
    found = False
    fromrow = minrow
    torow = maxrow
    targeted = ("",0)
    i = 0
    while (not found) :
        i += 1
        midrow = int(fromrow + (torow - fromrow)/2)
        targeted = query(db, f"select word, freq, rowid from words where rowid={midrow}")[0]
        #print(midrow, targeted)
        if word == targeted[0] or torow == fromrow:
            #print(targeted, torow, fromrow)
            found = True
        elif word > targeted[0]:
            torow = midrow 
        else:
            fromrow = midrow
    print(i)
    return targeted

def bisectall(db, word):
    
    minrow = query(db, "select rowid from words order by rowid asc limit 1")[0][0]
    maxrow = query(db, "select rowid from words order by rowid desc limit 1")[0][0]
    
    def bisect(db, word, minrow, maxrow):
        found = False
        fromrow = minrow
        torow = maxrow
        targeted = ("",0)
        res = []
        i = 0
        delta = 0
        limit = math.log2(maxrow + 1)
        while (not found) and (fromrow <= torow) and i < limit:
            i += 1
            midrow = int(fromrow + (torow - fromrow)/2)
            targeted = query(db, f"select word, freq, rowid from words where rowid={midrow}")[0]
            #print(midrow, targeted, torow, fromrow)
            if word == targeted[0]:
                res.append(targeted)
                print(targeted)
                found = True
                #print(i, targeted)
                left, li = bisect(db, word, fromrow, midrow -1)
                right, ri = bisect(db, word, midrow +1, torow )
                i = li + ri
                res += left
                res += right
            elif word > targeted[0]:
                torow = midrow
            else:
                fromrow = midrow 
            #print(i)
        return res, i
    
    return bisect(db, word, minrow, maxrow)


def bisectiterate(db, word):
    
    minrow = query(db, "select rowid from words order by rowid asc limit 1")[0][0]
    maxrow = query(db, "select rowid from words order by rowid desc limit 1")[0][0]
    
    def bisect(db, word, minrow, maxrow):
        found = False
        fromrow = minrow
        torow = maxrow
        targeted = ("",0)
        res = []
        i = 0
        delta = 0
        limit = math.log2(maxrow + 1)
        while (not found) and (fromrow <= torow) and i < limit:
            i += 1
            midrow = int(fromrow + (torow - fromrow)/2)
            targeted = query(db, f"select word, freq, rowid from words where rowid={midrow}")[0]
            #print(midrow, targeted, torow, fromrow)
            if word == targeted[0]:
                res.append(targeted)
                #print(targeted)
                found = True
                # print (i, targeted, fromrow, torow)
                
                leftcandidate = midrow - 1
                lcandidate = query(db, f"select word, freq, rowid from words where rowid={leftcandidate}")[0]
                while lcandidate[0] == word and leftcandidate >= fromrow:
                    #print(leftcandidate)
                    res.append(lcandidate)
                    leftcandidate -= 1
                    lcandidate = query(db, f"select word, freq, rowid from words where rowid={leftcandidate}")[0]
                    
                rightcandidate = midrow + 1
                rcandidate = query(db, f"select word, freq, rowid from words where rowid={rightcandidate}")[0]
                while rcandidate[0] == word and rightcandidate <= torow:
                    #print(rightcandidate)
                    res.append(rcandidate)
                    rightcandidate += 1
                    rcandidate = query(db, f"select word, freq, rowid from words where rowid={rightcandidate}")[0]
                    
            elif word > targeted[0]:
                torow = midrow
            else:
                fromrow = midrow 
            #print(i)
        return res
    
    return bisect(db, word, minrow, maxrow)

## Timing

Måler tid i sekunder, så Python ligger på 0.0X - se Julia ligger på 0.00X så ti ganger raskere 

In [23]:
d = time.time()
res = bisectiterate(db, "Nasjonalbiblioteket")
print("Tid i sekund", time.time() - d)
pd.DataFrame(res)

Tid i sekund 0.013101577758789062


,0,1,2
0,Nasjonalbiblioteket,419,171961292
1,Nasjonalbiblioteket,1108,171961291
2,Nasjonalbiblioteket,1301,171961290
3,Nasjonalbiblioteket,1357,171961289
4,Nasjonalbiblioteket,1372,171961288
5,Nasjonalbiblioteket,1816,171961287
6,Nasjonalbiblioteket,2276,171961286
7,Nasjonalbiblioteket,398,171961293
8,Nasjonalbiblioteket,39,171961294
9,Nasjonalbiblioteket,14,171961295


## Indeksert

I Python vil et oppslag på den indekserte tabellen tilsvare et fullt bisect-søk i Julia

In [22]:
%%time
with sqlite3.connect(db) as con:
    a = con.execute("select word, freq from hyphendata where word = 'Nasjonalbiblioteket' ").fetchall()
pd.DataFrame(a)

CPU times: user 4.11 ms, sys: 0 ns, total: 4.11 ms
Wall time: 2.76 ms


,0,1
0,Nasjonalbiblioteket,2
1,Nasjonalbiblioteket,12
2,Nasjonalbiblioteket,14
3,Nasjonalbiblioteket,39
4,Nasjonalbiblioteket,398
5,Nasjonalbiblioteket,419
6,Nasjonalbiblioteket,1108
7,Nasjonalbiblioteket,1301
8,Nasjonalbiblioteket,1357
9,Nasjonalbiblioteket,1372
